In [ ]:
from urllib.request import urlopen
f = urlopen('http://hanbit.co.kr')

In [ ]:
type(f)

In [ ]:
f.read()

In [ ]:
f.status

In [ ]:
f.getheader('Content-Type')

In [ ]:
encoding = f.info().get_content_charset(failobj='utf-8')

In [ ]:
text = f.read().decode('UTF-8')

In [ ]:
print(text)

In [ ]:
import sys
from urllib.request import urlopen

In [ ]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [ ]:
encoding = f.info().get_content_charset(failobj='utf-8')
print('encoding:', encoding, file=sys.stderr)

In [ ]:
text = f.read().decode(encoding)

In [ ]:
print(text)

In [ ]:
import re
import sys
from urllib.request import urlopen

In [ ]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [ ]:
bytes_content = f.read()

In [ ]:
scanned_text = bytes_content[:1024].decode('ascii', errors='replace')

In [ ]:
match = re.search(r'charset=["\']?([\w-]+)', scanned_text)

In [ ]:
if match:
    encoding = match.group(1)
else:
    encoding = 'utf-8'

In [ ]:
print('encoding:', encoding, file=sys.stderr)

In [ ]:
text = bytes_content.decode(encoding)

In [ ]:
print(text)

In [ ]:
test = "aa bb ab cb"
re.search(r'[acb]b', test)

In [ ]:
! pip list

In [ ]:
import re

In [ ]:
re.search(r'a.*c', 'abc12DEF')

In [ ]:
re.search(r'a.*d', 'abc123DEF')

In [ ]:
re.search(r'a.*d', 'abc123DEF', re.IGNORECASE)

In [ ]:
m = re.search(r'a([A-Z])c([a-z])', 'aDcd3DEcf')

In [ ]:
m.group(1)

In [ ]:
re.findall(r'\w{3,}', 'This is a pen')

In [ ]:
re.sub(r'\w{2,}', 'That', 'This is a pen')

In [ ]:
import sys
from urllib.request import urlopen
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

encoding = f.info().get_content_charset(failobj="utf-8")
print('encoding: ', encoding, file=sys.stderr)
text = f.read().decode(encoding)
print(text)
hf = open("./dp.html", 'w', encoding='utf-8')

for line in text:
    hf.write(line)
    
hf.close()


In [ ]:
# 2~11 scraping_re.py 정규표현식으로 스크레핑하기
import re
from html import unescape

with open('dp.html', encoding='utf-8') as f:
    html = f.read()
    
#print(html)
for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
    url = re.search(r'<a href="(.*?)">', partial_html).group(1)
    url = 'http://www.handbit.co.kr' + url
    title = re.sub(r'<.*?>', '', partial_html)
    title = unescape(title)
    print('url:', url)
    print('title:', title)
    print('------'*7)

In [ ]:
from xml.etree import ElementTree

tree = ElementTree.parse('rss.xml')

root = tree.getroot()

for item in root.findall('channel/item/description/body/location/data'):
    tm_ef = item.find('tmEf').text
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    print(tm_ef, tmn, tmx, wf)

In [ ]:
# 2.15 save

In [ ]:
import csv

with open('top_cities.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['rank', 'city', 'population'])
    writer.writerows([
        [1, '상하이', 24150000],
        [2, '카라치', 23500000],
        [3, '베이징', 21516000],
        [4, '텐진', 14722100],
        [5, '이스탄불', 14160467],
    ])
    

In [ ]:
# 2.16
import csv

with open('top_cities2.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, ['rank', 'city', 'population'])
    writer.writeheader()
    writer.writerows([
        {'rank':1, 'city':'상하이', 'population':24150000},
        {'rank':2, 'city':'카라치', 'population':23500000},
        {'rank':3, 'city':'베이징', 'population':21516000},
        {'rank':4, 'city':'텐진', 'population':14722100},
        {'rank':5, 'city':'이스탄불', 'population':14160467},
    ])

In [ ]:
import json

cities = [
    {'rank':1, 'city':'상하이', 'population':24150000},
    {'rank':2, 'city':'카라치', 'population':23500000},
    {'rank':3, 'city':'베이징', 'population':21516000},
    {'rank':4, 'city':'텐진', 'population':14722100},
    {'rank':5, 'city':'이스탄불', 'population':14160467},
]
print(json.dumps(cities))

In [ ]:
print(json.dumps(cities, ensure_ascii=False, indent=2))

In [ ]:
with open('top_cities.json', 'w') as f:
    json.dump(cities, f)

In [ ]:
import sqlite3

conn = sqlite3.connect('top_cities.db')
c = conn.cursor()

c.execute('DROP TABLE IF EXISTS cities')
c.execute('''
    CREATE TABLE cities (
        rank integer,
        city text,
        population integer
        )
        ''')
c.execute('INSERT INTO cities VALUES (?, ?, ?)', (1, '상하이', 24150000))
c.execute('INSERT INTO cities VALUES (:rank, :city, :population)', 
          {'rank':2, 'city': '카라치', 'population': 23500000})

c.executemany('INSERT INTO cities VALUES (:rank, :city, :population)', [
    {'rank':3, 'city':'베이징', 'population':21516000},
    {'rank':4, 'city':'텐진', 'population':14722100},
    {'rank':5, 'city':'이스탄불', 'population':14160467},
])

conn.commit()

c.execute('SELECT * FROM cities')
for row in c.fetchall():
    print(row)
    
conn.close

### 2.7 파이썬으로 스크레이핑하는 흐름
    * fetch(url)
       - 매개변수로 url을 받고 지정한 URL의 웹 페이지를 추출합니다.
    * scrape(html)
       - 매개변수로 html을 받고, 정규 표현식을 사용해 HTML에서 도서 정보를 추출합니다.
    * save(db_path, books)
       - 매개변수로 books라는 도서 목록을 받고, SQLite데이터베이스에 저장합니다.
       

In [ ]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape

def main():
    """
    메인 처리입니다.
    fetch(), scrape(), save() 함수를 호출합니다.
    """
    html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')
    books = scrape('html')
    save('books.db', books)

def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출합니다.
    웹 페이지의 인코딩 형식은 Content-Type 헤더를 기반으로 알아냅니다.
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    encoding = f.info().get_content_charset(failobj="utf-8")
    html = f.read().decode(encoding)
    return html

def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append({'url': url, 'title':title})
    return books

def save(db_path, books):
    """
    매개변수 books로 전달된 도서 목록을 SQLite 데이터베이스에 저장합니다.
    데이터베이스의 경로는 매개변수 dp_path로 지정합니다.
    반환값: None(없음)
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('DROP TABLE IF EXISTS books')
    c.execute('''
        CREATE TABLE books (
            title text,
            url text
        )
    ''')
    c.executemany('INSERT INTO books VALUES (:title, :url)', books)
    conn.commit()
    conn.close()
    
main()
    
    

In [ ]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape

def main():
    """
    메인 처리입니다.
    fetch(), scrape(), save() 함수를 호출합니다.
    """
    html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')
    books = scrape(html)
    save('books.db', books)

def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출합니다.
    웹 페이지의 인코딩 형식은 Content-Type 헤더를 기반으로 알아냅니다.
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식을 추출합니다.
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열을 디코딩합니다.
    html = f.read().decode(encoding)
    return html

def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출합니다.
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출합니다.
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목을 추출합니다.
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append({'url': url, 'title': title})
    
    return books

def save(db_path, books):
    """
    매개변수 books로 전달된 도서 목록을 SQLite 데이터베이스에 저장합니다.
    데이터베이스의 경로는 매개변수 dp_path로 지정합니다.
    반환값: None(없음)
    """
    # 데이터베이스를 열고 연결을 확립합니다.
    conn = sqlite3.connect(db_path)
    # 커서를 추출합니다.
    c = conn.cursor()
    # execute() 메서드로 SQL을 실행합니다.
    # 스크립트를 여러 번 실행할 수 있으므로 기존의 books 테이블을 제거합니다.
    c.execute('DROP TABLE IF EXISTS books')
    # books 테이블을 생성합니다.
    c.execute('''
        CREATE TABLE books (
            title text,
            url text
        )
    ''')
    # executemany() 메서드를 사용하면 매개변수로 리스트를 지정할 수 있습니다.
    c.executemany('INSERT INTO books VALUES (:title, :url)', books)
    # 변경사항을 커밋(저장)합니다.
    conn.commit()
    # 연결을 종료합니다.
    conn.close()

# python 명령어로 실행한 경우 main() 함수를 호출합니다.
# 이는 모듈로써 다른 파일에서 읽어 들였을 때 main() 함수가 호출되지 않게 하는 것입니다.
# 파이썬 프로그램의 일반적인 작성 방식입니다.
if __name__ == '__main__':
    main()

In [ ]:
! pip list

In [ ]:
!pip install requests

In [ ]:
import requests
r = requests.get('http://hanbit.co.kr')
type(r)

In [ ]:
r.status_code

In [ ]:
r.headers['content-type']

In [ ]:
r.encoding

In [ ]:
r.text

In [ ]:
r = requests.get('https://jsonplaceholder.typicode.com/todos/1')
r.text

In [ ]:
r.json()

In [ ]:
r = requests.post('http://httpbin.org/post', data={'key1':'value'})

In [ ]:
r

In [ ]:
import lxml.html

In [ ]:
tree = lxml.html.parse('full_book_list.html')

In [ ]:
tree

In [ ]:
html = tree.getroot()

for a in html.xpath('//a'):
    print(a.get('href'), a.text)
    

In [ ]:
html = lxml.html.fromstring('''
<html>
    <head><title>온라인 과일 가게</title></head>
    <body>
        <h1 id="main">오늘의 과일</h1>
        <ul>
            <li>사과</li>
            <li class="featured">귤</li>
            <li>포도</li>
        </ul>
    </body>
</html>
''')

In [ ]:
type(html)

In [ ]:
html.xpath('//li')

In [ ]:
for a in html.xpath('//li'):
    print(a.text)

In [ ]:
h1 = html.xpath('//h1')[0]

In [ ]:
h1.text

h1

In [ ]:
h1.get('id')

In [ ]:
h1.attrib

## 웹 서버에 요청하고 응답받기

In [ ]:
import requests
url = "https://www.python.org/"
resp = requests.get(url)
print(resp)

url2 = "https://www.python.org/1"
resp = requests.get(url2)
print(resp)

## 웹 페이지 소스코드 확인하기

In [ ]:
import requests
url = "https://www.python.org"
resp = requests.get(url)
print(resp.__dir__())
print('---'*20)
print(resp.headers)
print('---'*20)
print(resp.status_code)
print('---'*20)
#print(resp._content)
print('---'*20)
print(resp.url)
print('---'*20)
print(resp.text)

/robots.txt->사이트 크롤링 허용 부분 정보 확인

## 로봇 배제 표준(robots.txt)

In [ ]:
import requests
urls = ['https://www.naver.com/', 'https://www.python.org/']
filename = 'robots.txt'

for url in urls:
    file_path = url + filename
    resp = requests.get(file_path)
    #print(file_path)
    print(resp.text)
    print('---'*20)

## Beautifulsoup 객체만들기

In [ ]:
pip list

In [ ]:
soup = BeautifulSoup('<b class="boldest">Extremely bold</b>')
tag = soup.b
tag['class']
tag.attrs

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Seoul_Metropolitan_Subway'
resp = requests.get(url)
html_src = resp.text
#print(html_src)
soup = BeautifulSoup(html_src, 'html.parser')
print('title 태그 요소: ', soup.title)
print('title 태그 이름: ', soup.title.name)
print('title 태그 문자열: ', soup.title.string)
first_img = soup.select('div > table > tbody > tr > td > a > img')
print(first_img)
print('---'*20)
first_img2 = soup.find(name='img', attrs={'alt':'Seoul-metro-2009-20180916-103548.jpg'})
print(first_img2)

## 웹 문서의 그림 이미지 파일을 PC에 저장하기-완료. 다운로드 잘 됨.

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Seoul_Metropolitan_Subway'
resp = requests.get(url)
html_src = resp.text

soup = BeautifulSoup(html_src, 'html.parser')

target_image = soup.find(name='img', attrs={'alt': 'Seoul-metro-2009-20180916-103548.jpg'})
print('HTML 요소: ', target_image)
print('--'*25)
targ_img_src = target_image.get('src')
print('이미지 파일 경로: ', targ_img_src)
print('--'*25)
target_img_resp = requests.get('http:'+ targ_img_src)
print(target_img_resp)
print('--'*20)
out_file_path = "download_image.jpg"

with open(out_file_path, 'wb') as out_file:
    out_file.write(target_img_resp.content)
    print('이미지 파일로 저장하였습니다.')

## 웹 문서에 포함된 모든 하이퍼링크 추출하기

In [ ]:
import requests, re
from bs4 import BeautifulSoup
url = 'https://en.wikipedia.org/wiki/Seoul_Metropolitan_Subway'
resp = requests.get(url)
html_src = resp.text
soup = BeautifulSoup(html_src, 'html.parser')

links = soup.find_all("a")
print(type(links))
print("하이퍼링크의 개수: ", len(links))
print('--'*20)
print('첫 3개의 원소: ', links[:3])
print('--'*20)
wiki_links = soup.find_all(name='a', href=re.compile("/wiki/"), limit=3)
print('/wiki/문자열이 포함된 하이퍼링크: ', wiki_links)
print('--'*20)
external_links = soup.find_all(name='a', attrs={'class':"external text"}, limit=3)
print('class 속성으로 추출한 하이퍼링크: ', external_links)

## CSS Selector 활용하기 1

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Seoul_Metropolitan_Subway'
resp = requests.get(url)
html_src = resp.text

soup = BeautifulSoup(html_src, 'html.parser')

subway_image = soup.select('div.mw-parser-output > table > tbody > tr > td > a > img')
subway_image1 = soup.select('#mw-content-text > div.mw-parser-output > table:nth-child(5) > tbody > tr:nth-child(2) > td > a > img')
# #mw-content-text > div.mw-parser-output > table:nth-child(5) > tbody > tr:nth-child(2) > td > a > img
# div.mw-parser-output > table > tbody > tr > td > a > img
print(subway_image)
print('--'*20)
#print(subway_image1[0])

subway_image2 = soup.select('a > img')
print(len(subway_image2))

for i in range(5):
    print(subway_image2[i])
    print()

## CSS Selector 활용하기 2

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Seoul_Metropolitan_Subway'
resp = requests.get(url)
html_src = resp.text
soup = BeautifulSoup(html_src, 'html.parser')

links = soup.select('a')
print(len(links))
print()
print(links[:3])
print()
external_links = soup.select('a[class="external text"]')
print(external_links[:3])
print()

external_links = soup.select('a.external text')
#빈공간은 인식 못함
print(external_links[:3])
print()

jump_links = soup.select('a[class="mw-jump-link"]')
print(jump_links[:3])
print()

id_selector = soup.select('#siteNotice')
print('id_selector: ', id_selector)
print()
id_selector2 = soup.select('div#siteNotice')
print('id_selector2: ', id_selector2)
print()
class_selector = soup.select('span.mw-headline')
print('class_selector: ', class_selector)

## 구글 뉴스 클리핑하기1

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
# keyword_input = "아파트"

# keyword = urllib.parse.quote(keyword_input)
# print('파이썬 문자열을 URL 코드로 변환: ', keyword)

base_url = "https://news.google.com"
# search_url = base_url + "/search?q=" + keyword +"&hl=ko&gl=KR&ceid=KR%3Ako"
# print("검색어와 조합된 URL: ", search_url)

# resp = requests.get(search_url)
# html_src = resp.text
# soup = BeautifulSoup(html_src, 'html.parser')
# #print(soup)


# news_items = soup.select('div[class="xrnccd"]')

# #print(len(news_items))
#n = 0
#for i in news_items:
    #n += 1
    #print(f'{n}번째: {i}')
    #print()
n = 0    
# for item in news_items:
#     n += 1
#     news_title = item.find('a', attrs={'class': 'DY5T1d'}).getText()
#     print(f'{n}번째: {news_title}')
#     news_agency = item.find('a', attrs={'class': 'wEwyrc'}).getText()
#     print(news_agency)
    
#     news_reporting = item.find('time', attrs={'class': 'WW6dff'}).text
#     print(news_reporting)
    
#     news_reporting = item.find('time', attrs={'class': 'WW6dff'})
#     news_reporting_datetime = news_reporting.get('datetime').split('T')
#     news_reporting_date = news_reporting_datetime[0]
#     news_reporting_time = news_reporting_datetime[1][:-1]
#     print(news_reporting_date, news_reporting_time)
#     print()

def google_news_clipping(keyword_input, limit=5):
    
    keyword = urllib.parse.quote(keyword_input)

    url = base_url + "/search?q=" + keyword +"&hl=ko&gl=KR&ceid=KR%3Ako"
    
    resp = requests.get(url)
    html_src = resp.text
    soup = BeautifulSoup(html_src, 'html.parser')
    
    news_items = soup.select('div[class="xrnccd"]')
    
    links = []; titles=[]; agencies=[]; reporting_dates=[]; reporting_times=[];
    
    for item in news_items[:limit]:
        link = item.find('a', attrs={'class': 'VDXfz'}).get('href')
        news_link = base_url + link[1:]
        links.append(news_link)
        
        news_title = item.find('a', attrs={'class': 'DY5T1d'}).getText()
        titles.append(news_title)
        
        news_agency = item.find('a', attrs={'class': 'wEwyrc'}).text
        agencies.append(news_agency)
        
        news_reporting = item.find('time', attrs={'class':'WW6dff'})
        news_reporting_datetime = news_reporting.get('datetime').split('T')
        
        news_reporting_date = news_reporting_datetime[0]
        news_reporting_time = news_reporting_datetime[1][:-1]
        reporting_dates.append(news_reporting_date)
        reporting_times.append(news_reporting_time)
    
    result = {'link': links, 'title':titles, 'agency':agencies,\
             'date': reporting_dates, 'time': reporting_times}
    
    return result

def display(news, num):
    
    for i in range(num):
        print(news['title'][i])
        print(news['agency'][i])
        print(news['date'][i])
        print(news['time'][i])
        print(news['link'][i])
        print()
        print()

search_word = input("검색어를 입력하세요: ")
# print("search_word: ", search_word)
        
news = google_news_clipping(search_word, 5)
display(news,5)
    

In [ ]:
#https://news.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&gl=KR&ceid=KR%3Ako

In [ ]:
#yDmH0d > c-wiz:nth-child(31) > div > div.FVeGwb.CVnAc.Haq2Hf.bWfURe > div.ajwQHc.BL5WZb.RELBvb > div > main > c-wiz > div.lBwEZb.BL5WZb.xP6mwf > div.NiLAwe.mi8Lec.gAl5If.jVwmLb.Oc0wGc.R7GTQ.keNKEd.j7vNaf.nID9nc > div > div > article > h3

In [ ]:
신문사: #yDmH0d > c-wiz:nth-child(31) > div > div.FVeGwb.CVnAc.Haq2Hf.bWfURe > div.ajwQHc.BL5WZb.RELBvb > div > main > c-wiz > div.lBwEZb.BL5WZb.xP6mwf > div.NiLAwe.mi8Lec.gAl5If.jVwmLb.Oc0wGc.R7GTQ.keNKEd.j7vNaf.nID9nc > div > div > article > div > div > a

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(3)
driver.get("https://www.danawa.com")

login = driver.find_element_by_css_selector('#danawa_header > div > div > div.main-header__banner > div.main-header__user > div:nth-child(5) > a')
#danawa_header > div > div > div.main-header__banner > div.main-header__user > div:nth-child(5) > a

print('HTML 요소: ', login)
print('태그 이름: ', login.tag_name)
print('문자열: ', login.text)
print('href 속성: ', login.get_attribute('href'))
driver.implicitly_wait(5)
login.click()


driver.implicitly_wait(10)
# naver_login = driver.find_element_by_css_selector('#danawa-member-login-snsButton-naver')
# driver.implicitly_wait(3)

# naver_login.click()

my_id = "yjk1322"
my_pw = "alcjsdbwls4132*"


driver.find_element_by_id('danawa-member-login-input-id').send_keys(my_id)
driver.implicitly_wait(3)
driver.find_element_by_name('password').send_keys(my_pw)
driver.find_element_by_css_selector('button.btn_login').click()


wishlist = driver.find_element_by_css_selector('#danawa_header > div > div > div.main-header__banner > div.main-header__user > div:nth-child(4) > a')
wishlist.click()
driver.implicitly_wait(2)
html_src = driver.page_source
# print(html_src)
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(html_src, 'html.parser')


wish_table = soup.select('table[class="tbl wish_tbl"]')[0]
# print(wish_table)
# print(type(wish_table))
wish_items = wish_table.select('tbody tr')
# print(wish_items)

for item in wish_items:
    title = item.find('div', {'class': 'tit'}).text
    price = item.find('span', {'class': 'price'}).text
    link = item.find('a', href=re.compile("http://prod.danawa.com/info/")).get('href')
    print('title: ', title)
    print('price: ', price.split())
    print('link: ', link)
    print('--'*20)
    



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\PNU\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\PNU\AppData\Local\Temp/ipykernel_2328/3837665581.py:12: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  login = driver.find_element_by_css_selector('#danawa_header > div > div > div.main-header__banner > div.main-header__user > div:nth-child(5) > a')


HTML 요소:  <selenium.webdriver.remote.webelement.WebElement (session="fd007584a05489f1d47cb8fb7abbab4a", element="6c5eaf9a-f04d-42c7-8b0f-521da2a5762f")>
태그 이름:  a
문자열:  로그인
href 속성:  https://auth.danawa.com/login?url=http%3A%2F%2Fwww.danawa.com%2F


C:\Users\PNU\AppData\Local\Temp/ipykernel_2328/3837665581.py:33: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('danawa-member-login-input-id').send_keys(my_id)
C:\Users\PNU\AppData\Local\Temp/ipykernel_2328/3837665581.py:35: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name('password').send_keys(my_pw)
C:\Users\PNU\AppData\Local\Temp/ipykernel_2328/3837665581.py:36: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('button.btn_login').click()
C:\Users\PNU\AppData\Local\Temp/ipykernel_2328/3837665581.py:39: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  wishlist = driver.find_element_by_css_selector('#danawa

title:  삼성전자 비스포크 RF85A911155
price:  ['1,684,850원']
link:  http://prod.danawa.com/info/?pcode=14710061
title:  QCY T13 (정품)
price:  ['14,410원']
link:  http://prod.danawa.com/info/?pcode=15071006
title:  LG전자 오브제컬렉션 코드제로 ThinQ A9S AO9571 (카밍베이지)
price:  ['991,160원']
link:  http://prod.danawa.com/info/?pcode=13327520


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(30)
driver.get("https://ecos.bok.or.kr/jsp/vis/keystat/#/key")

excel_download = driver.find_element_by_css_selector('img[alt="download"]')
driver.implicitly_wait(3)
excel_download.click()
time.sleep(5)
print("파일 다운로드 실행...")
# driver.close()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\PNU\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\PNU\AppData\Local\Temp/ipykernel_2328/2895235658.py:13: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  excel_download = driver.find_element_by_css_selector('img[alt="download"]')


파일 다운로드 실행...


In [ ]:
pip install selenium

In [ ]:
pip install webdriver_manager

In [ ]:
pip install nbconvert, pandoc

In [38]:
import re; import sys; print(sys.modules['re'])

<module 're' from 'C:\\Anaconda3\\lib\\re.py'>
